# Código pincipal

In [1]:
# 

import time
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys # Necessário para enviar caracteres especiais no comando .send_keys()
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Definir site
Site = "https://nvd.nist.gov/vuln/search"

# Abrir o site
driver.get(Site)


### Funções
    
def  getListFromElements(Caminho):    
    '''
    Retorna uma lista com o texto dos elementos encontrados no caminho(xpath) 
    
    '''
    Elemento = driver.find_elements(By.XPATH , Caminho)
    Lista = []
    for item in Elemento:
        Lista.append(item.text)
    return Lista

def isLastPage():  
    '''
    Idendifica existência de página adicional e segue para próxima caso exista
    
    '''
    Navegador = driver.find_elements(By.XPATH, '//*[@id="row"]/div/div/nav/ul/li/a')
    Saida = True
    time.sleep(1) 
    for item in Navegador:
        if item.text == '>':
            print('Abrindo próxima página...')
            item.click()
            Saida = False
            break
        else:
            Saida = True
            
    return Saida   

def itensSeveros(Severidades,SeveridadeCritica=8):
    itens = 0
    for severidade in Severidades:
        teste = (severidade >= SeveridadeCritica)
        if teste:
            itens += 1
    return itens
    
def EnviarEmail():
    return True

def GerarPlanilha():
    return True

def colorize(text,color):
    '''
    muda a cor do texto 'text' na cor 'color'
    '''
    text = color + text + colorReset
    return text


print('Declarando varáveis')    
colorReset = "\033[0;0m"
colorRed   = "\033[1;31m"
colorGrenn = "\033[0;32m"
colorBlue  = "\033[1;34m"
colorCyan = "\033[1;36m"
fontBold    = "\033[;1m"
fontReverse = "\033[;7m"
colorReset = "\033[0;0m"
colorRed   = "\033[1;31m"
colorGrenn = "\033[0;32m"
colorBlue  = "\033[1;34m"
colorCyan = "\033[1;36m"
fontBold    = "\033[;1m"
fontReverse = "\033[;7m"

Pesquisado = input('Software a ser pesquisado (T = teste): ')
if Pesquisado == 'T':
    Pesquisado = 'log4j'
    DataInicial = '01012022'
    DataFinal = '05012022'
    emailCadastrado = 'alerta_cve@teste.com'
    nomeUsuario = 'Teste'
else:
    DataInicial = input('Data inicial(mm/dd/aaaa-mmddaaaa): ')
    DataFinal = input('Data final(mm/dd/aaaa-mmddaaaa): ')
    nomeUsuario = 'Teste'
    emailCadastrado = 'alerta_cve@teste.com'

SeveridadeCritica = 8
Severidades = [1,2,3,4,8,2.5,9,10]
lstCVE = []
lstCVSS_v20 = []
lstCVSS_v31 = []
lstDescr = []
lstPubli = []
sistema = []
Links = []
Referencias = []
Configs = []

print('Definindo xpaths')
XpathCVE = '//*/tbody/tr/th/strong/a'
XpathCVSS_v20 = '//*/tbody/tr/td[2]/span[2]'
XpathCVSS_v31 = '//*/tbody/tr/td[2]/span[1]'
XpathDescr = '//*/tbody/tr/td[1]/p'
XpathPubli = '//*/table/tbody/tr/td[1]/span'
xpathReferencia = '//*/table/tbody/tr/td/div/div[1]/div/div[1]/table/tbody/tr/td[1]/a'
xpathConfig = '//*[@id="config-div-1"]/table/tbody/tr/td[1]/b'

print('Ativando busca avançada')
AdvavancedSearch = driver.find_element(value='SearchTypeAdvanced')
AdvavancedSearch.click()

print('Encontrando elementos na página')
# Keyword Search //*[@id="Keywords"]
Keyword = driver.find_element(value="Keywords")
Keyword.clear()

# Encontrar os elementos Data inicial e Data final
StartDate = driver.find_element(value='published-start-date')
StartDate.clear()

EndDate = driver.find_element(value='published-end-date')
EndDate.clear()

print('Inserindo configurações de busca')
Keyword.send_keys(Pesquisado)
StartDate.send_keys(DataInicial)
EndDate.send_keys(DataFinal)

print('Iniciando busca por vulnerabilidades em '+ colorize(Pesquisado,colorGrenn))
# Clicar no botão "Search"
SubmitSearch = driver.find_element(value='vuln-search-submit')
SubmitSearch.click()
time.sleep(2)

LastPage = False        
while not(LastPage):
    lstCVE += getListFromElements(XpathCVE)
    lstCVSS_v20 += getListFromElements(XpathCVSS_v20)
    lstCVSS_v31 += getListFromElements(XpathCVSS_v31)
    lstDescr += getListFromElements(XpathDescr)
    lstPubli += getListFromElements(XpathPubli)
    LastPage = isLastPage()
    
if len(lstCVE) == 0:
    print('Na busca por ' + colorize(Pesquisado,colorBlue) + ', nenhuma vulnerabilidade foi encontrada')
elif len(lstCVE) == 1:
    print(len('>>> Uma nova vulnerabilidade encontrada'))
else:
    print('>>>', len(lstCVE), 'novas vulnerabilidades foram encontradas')

   
        
for i in lstCVE:
    Links.append('https://nvd.nist.gov/vuln/detail/'+i)
    sistema.append(Pesquisado)
    
contador = len(Links)
print('Buscando informações de referencia e configuração para cada vulnerabilidade...')
for link in Links:
    driver.get(link)
    
    lstConfig = getListFromElements(xpathConfig)
    
    lstReferencia = getListFromElements(xpathReferencia)
    
    print(contador,'|',end=' ')
    Referencia = []    
    for r in range(len(lstReferencia)):
        Referencia.append(lstReferencia[r])
    Referencias.append(Referencia)
    
    Config = []    
    for l in range(len(lstConfig)):
        Config.append(lstConfig[l])
    Configs.append(Config)
    
    contador = contador - 1
    

if len(lstCVE) > 0 and itensSeveros(Severidades, SeveridadeCritica):

    print("\n\n "+colorize("!!!ATENÇÃO!!!",colorRed) , itensSeveros(Severidades, SeveridadeCritica),  "vulnerabilidade(s) com severidade considerada crítica encontrada!\n\n")
    print('já estou enviando relatório para o e-mail: '+ colorize(emailCadastrado,colorBlue) + ' cadastrado\n')

else:
    print("\nNenuma vulnerabilidade com severidade considerada crítica foi encontrada!\n") 
    
    
print('Adicionando informações ao dicionário')
Tabela={}
Tabela.update({'CVE': lstCVE})
Tabela.update({'CVSS_v20': lstCVSS_v20})
Tabela.update({'CVSS_v31': lstCVSS_v31})
Tabela.update({'Descrição': lstDescr})
Tabela.update({'Publicação': lstPubli})    
Tabela.update({'Link':Links})
Tabela.update({'Sistema':sistema})
Tabela.update({'Referencias':Referencias})
Tabela.update({'Configuração':Configs})

print("Aqui está, tudo o que encontrei até agora:\n")
pprint(Tabela)
print('Encerrando o navegador.')
time.sleep(5)
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Lenovo\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


Declarando varáveis
Software a ser pesquisado (T = teste): T
Definindo xpaths
Ativando busca avançada
Encontrando elementos na página
Inserindo configurações de busca
Iniciando busca por vulnerabilidades em log4j
>>> 9 novas vulnerabilidades foram encontradas
Buscando informações de referencia e configuração para cada vulnerabilidade...
9 | 8 | 7 | 6 | 5 | 4 | 3 | 2 | 1 | 

 !!!ATENÇÃO!!! 3 vulnerabilidade(s) com severidade considerada crítica encontrada!


já estou enviando relatório para o e-mail: alerta_cve@teste.com cadastrado

Adicionando informações ao dicionário
Aqui está, tudo o que encontrei até agora:

{'CVE': ['CVE-2022-0070',
         'CVE-2021-3100',
         'CVE-2022-24818',
         'CVE-2022-23848',
         'CVE-2022-21704',
         'CVE-2022-23307',
         'CVE-2022-23305',
         'CVE-2022-23302',
         'CVE-2021-44530'],
 'CVSS_v20': ['V2.0: 7.2 HIGH',
              'V2.0: 7.2 HIGH',
              'V2.0: 7.5 HIGH',
              'V2.0: 7.5 HIGH',
          

In [2]:
import pandas as pd

def planilha(Tabela):
    print('Iniciando verificação dos dados...')
    for i in Tabela:
        datas = {}
        severidade = {}
        severidade_v3 = {}
        datas,horas = Tabela.get('Publicação')[1].split(sep=";")
        versao,severidade,nivel = Tabela.get('CVSS_v20')[1].split(sep=" ")
        versao_v3,severidade_v3,nivel_v3 = Tabela.get('CVSS_v31')[1].split(sep=" ")
        datas = {'Publicação': datas}
        severidade = {'CVSS_v20': float(severidade)}
        severidade_v3 = {'CVSS_v31': float(severidade_v3)}
        #print(datas)
        #print(severidade)
        #print(severidade_v3)
    
    Tabela.update(datas)
    Tabela.update(severidade)
    Tabela.update(severidade_v3)
    
    df = pd.DataFrame.from_dict(Tabela, orient ='columns') #dicionario para dataframe
    df = df.rename(columns={'CVE_ID':'CVE','CVSS_v31':'Severity_v3','CVSS_v20':
                        'Severity_v2','Descrição':'Current Description','Publicação':'NVD','Sistema':'Software/Sistema','Referencias':'References','Configuração':'Known'}) #renomear as colunas no dataframe
    
    df = df[['Software/Sistema',
                   'CVE',
                   'Current Description',
                  'Severity_v2','Severity_v3',
                  'References',
                  'Known',
                  'NVD',
                  'Link']] #organizar a posição das colunas
    print('Conversão para planilha finalizada!')
    df.to_excel('Planilha.xlsx')
    print('Planilha criada com sucesso!')
    return df

In [3]:
df = planilha(Tabela)
display(df)

Iniciando verificação dos dados...
Conversão para planilha finalizada!
Planilha criada com sucesso!


,Software/Sistema,CVE,Current Description,Severity_v2,Severity_v3,References,Known,NVD,Link
0,log4j,CVE-2022-0070,Incomplete fix for CVE-2021-3100. The Apache L...,7.2,8.8,[https://alas.aws.amazon.com/cve/html/CVE-2022...,[ cpe:2.3:a:amazon:hotpatch:*:*:*:*:*:log4j:*:...,"Abril 19, 2022",https://nvd.nist.gov/vuln/detail/CVE-2022-0070
1,log4j,CVE-2021-3100,The Apache Log4j hotpatch package before log4j...,7.2,8.8,[https://alas.aws.amazon.com/AL2/ALAS-2021-173...,[ cpe:2.3:a:amazon:log4jhotpatch:*:*:*:*:*:*:*...,"Abril 19, 2022",https://nvd.nist.gov/vuln/detail/CVE-2021-3100
2,log4j,CVE-2022-24818,GeoTools is an open source Java library that p...,7.2,8.8,[https://github.com/geotools/geotools/commit/4...,"[ cpe:2.3:a:geotools:geotools:*:*:*:*:*:*:*:*,...","Abril 19, 2022",https://nvd.nist.gov/vuln/detail/CVE-2022-24818
3,log4j,CVE-2022-23848,"In Alluxio before 2.7.3, the logserver does no...",7.2,8.8,[https://www.alluxio.io/download/releases/allu...,[ cpe:2.3:a:alluxio:alluxio:*:*:*:*:*:*:*:*],"Abril 19, 2022",https://nvd.nist.gov/vuln/detail/CVE-2022-23848
4,log4j,CVE-2022-21704,log4js-node is a port of log4js to node.js. In...,7.2,8.8,[https://github.com/log4js-node/log4js-node/bl...,[ cpe:2.3:a:log4js_project:log4js:*:*:*:*:*:no...,"Abril 19, 2022",https://nvd.nist.gov/vuln/detail/CVE-2022-21704
5,log4j,CVE-2022-23307,CVE-2020-9493 identified a deserialization iss...,7.2,8.8,[https://lists.apache.org/thread/rg4yyc89vs3dw...,"[ cpe:2.3:a:apache:chainsaw:*:*:*:*:*:*:*:*, ...","Abril 19, 2022",https://nvd.nist.gov/vuln/detail/CVE-2022-23307
6,log4j,CVE-2022-23305,"By design, the JDBCAppender in Log4j 1.2.x acc...",7.2,8.8,[http://www.openwall.com/lists/oss-security/20...,[ cpe:2.3:a:apache:log4j:*:*:*:*:*:*:*:*],"Abril 19, 2022",https://nvd.nist.gov/vuln/detail/CVE-2022-23305
7,log4j,CVE-2022-23302,JMSSink in all versions of Log4j 1.x is vulner...,7.2,8.8,[http://www.openwall.com/lists/oss-security/20...,[ cpe:2.3:a:apache:log4j:*:*:*:*:*:*:*:*],"Abril 19, 2022",https://nvd.nist.gov/vuln/detail/CVE-2022-23302
8,log4j,CVE-2021-44530,An injection vulnerability exists in a third-p...,7.2,8.8,[https://community.ui.com/releases/Security-Ad...,[ cpe:2.3:a:ui:unifi_network_controller:*:*:*:...,"Abril 19, 2022",https://nvd.nist.gov/vuln/detail/CVE-2021-44530


In [4]:

from openpyxl import Workbook, load_workbook
# Para alterar os estilos de formatação das células
from openpyxl.styles import PatternFill, Font, Alignment

def organizar(df):
    # Carregar dados para variável
    wb = load_workbook('Planilha.xlsx')
    # Escolhe active sheet
    ws = wb.active
    # Deleta primeira coluna, que é somente índice
    ws.delete_cols(1)
    # Cabeçalho em negrito e fundo azul
    # Fill parameters
    my_fill = PatternFill(start_color='5399FF', 
                       end_color='5399FF',
                       fill_type='solid')
    # Bold Parameter
    my_font = Font(bold=True) ##tamanho e fonte
    # Formata o cabeçalho
    my_header = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
    for cell in my_header:
        ws[cell].fill = my_fill
        ws[cell].font = my_font
    # Formata as colunas e linhas
    ws.column_dimensions['A'].width = 20
    ws.column_dimensions['B'].width = 20
    ws.column_dimensions['C'].width = 80
    ws.column_dimensions['D'].width = 20
    ws.column_dimensions['E'].width = 20
    ws.column_dimensions['F'].width = 40
    ws.column_dimensions['G'].width = 40
    ws.column_dimensions['H'].width = 40
    ws.column_dimensions['I'].width = 40
    for row in ws.iter_rows():  
        for cell in row:      
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    #alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    #cell = ws['C2']
    #ws.alignment = alignment
    # Salva o arquivo
    print('Planilha configurada')
    wb.save('Planilha.xlsx')
    return 'Planilha salva'

In [5]:
df = organizar(df)
display(df)

Planilha configurada


'Planilha salva'